In [ ]:
# Installing required packages
!pip install pyspark
!pip install findspark

In [ ]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

In [ ]:
from pyspark.sql import SQLContext

In [ ]:
spark = SparkSession \
    .builder \
    .appName("ML_Regression_Example1") \
    .getOrCreate()

# Create Spark Context

In [ ]:
sc = spark.sparkContext


sqlContext = SQLContext(sc)

/usr/local/lib/python3.11/dist-packages/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [ ]:
from google.colab import files
uploaded = files.upload()


Saving daily_weather.csv to daily_weather (1).csv


In [ ]:
file='daily_weather.csv'

df = sqlContext.read.load(file,
                          format='com.databricks.spark.csv',
                          header='true',inferSchema='true')

In [ ]:
df.printSchema()
df.show()

root
 |-- air_pressure_9am: double (nullable = true)
 |-- air_temp_9am: double (nullable = true)
 |-- avg_wind_direction_9am: double (nullable = true)
 |-- avg_wind_speed_9am: double (nullable = true)
 |-- max_wind_direction_9am: double (nullable = true)
 |-- max_wind_speed_9am: double (nullable = true)
 |-- rain_accumulation_9am: double (nullable = true)
 |-- rain_duration_9am: double (nullable = true)
 |-- relative_humidity_9am: double (nullable = true)
 |-- relative_humidity_3pm: double (nullable = true)

+-----------------+------------------+----------------------+------------------+----------------------+------------------+---------------------+-----------------+---------------------+---------------------+
| air_pressure_9am|      air_temp_9am|avg_wind_direction_9am|avg_wind_speed_9am|max_wind_direction_9am|max_wind_speed_9am|rain_accumulation_9am|rain_duration_9am|relative_humidity_9am|relative_humidity_3pm|
+-----------------+------------------+----------------------+-----------

In [ ]:
df.columns

['number',
 'air_pressure_9am',
 'air_temp_9am',
 'avg_wind_direction_9am',
 'avg_wind_speed_9am',
 'max_wind_direction_9am',
 'max_wind_speed_9am',
 'rain_accumulation_9am',
 'rain_duration_9am',
 'relative_humidity_9am',
 'relative_humidity_3pm']

In [ ]:
df.printSchema()

root
 |-- number: integer (nullable = true)
 |-- air_pressure_9am: double (nullable = true)
 |-- air_temp_9am: double (nullable = true)
 |-- avg_wind_direction_9am: double (nullable = true)
 |-- avg_wind_speed_9am: double (nullable = true)
 |-- max_wind_direction_9am: double (nullable = true)
 |-- max_wind_speed_9am: double (nullable = true)
 |-- rain_accumulation_9am: double (nullable = true)
 |-- rain_duration_9am: double (nullable = true)
 |-- relative_humidity_9am: double (nullable = true)
 |-- relative_humidity_3pm: double (nullable = true)



In [ ]:
df.describe().toPandas().transpose()

,0,1,2,3,4
summary,count,mean,stddev,min,max
number,1095,547.0,316.24357700987383,0,1094
air_pressure_9am,1092,918.8825513138094,3.184161180386833,907.9900000000024,929.3200000000012
air_temp_9am,1090,64.93300141287072,11.175514003175877,36.752000000000685,98.90599999999992
avg_wind_direction_9am,1091,142.2355107005759,69.13785928889189,15.500000000000046,343.4
avg_wind_speed_9am,1092,5.50828424225493,4.5528134655317185,0.69345139999974,23.554978199999763
max_wind_direction_9am,1092,148.95351796516923,67.23801294602953,28.89999999999991,312.19999999999993
max_wind_speed_9am,1091,7.019513529175272,5.598209170780958,1.1855782000000479,29.84077959999996
rain_accumulation_9am,1089,0.20307895225211126,1.5939521253574893,0.0,24.01999999999907
rain_duration_9am,1092,294.1080522756142,1598.0787786601481,0.0,17704.0


In [ ]:
df.describe('air_pressure_9am').show()

+-------+-----------------+
|summary| air_pressure_9am|
+-------+-----------------+
|  count|             1092|
|   mean|918.8825513138094|
| stddev|3.184161180386833|
|    min|907.9900000000024|
|    max|929.3200000000012|
+-------+-----------------+



In [ ]:
len(df.columns)

11

In [ ]:
df.count()

1095

In [ ]:
df2 = df.na.drop(subset=['air_pressure_9am'])

In [ ]:
df2.count()

1092

In [ ]:
df2.stat.corr("rain_accumulation_9am", "rain_duration_9am")

0.7298253479609021

#  Regression

In [ ]:
from pyspark.sql import DataFrameNaFunctions
from pyspark.ml.feature import VectorAssembler

In [ ]:
df.count(), len(df.columns)

(1095, 11)

drop number column

In [ ]:
df = df.drop('number')

In [ ]:
df = df.na.drop()

In [ ]:
df.count(), len(df.columns)

(1064, 10)

In [ ]:
df.columns

['air_pressure_9am',
 'air_temp_9am',
 'avg_wind_direction_9am',
 'avg_wind_speed_9am',
 'max_wind_direction_9am',
 'max_wind_speed_9am',
 'rain_accumulation_9am',
 'rain_duration_9am',
 'relative_humidity_9am',
 'relative_humidity_3pm']

In [ ]:
#featureColumns =df.columns[:-2]
featureColumns = ['air_pressure_9am','max_wind_speed_9am','max_wind_direction_9am']

In [ ]:
featureColumns

['air_pressure_9am', 'max_wind_speed_9am', 'max_wind_direction_9am']

uses a single Vector to store the features instead of using 2 seperate columns

In [ ]:
assembler = VectorAssembler(inputCols=featureColumns, outputCol="features")
assembled = assembler.transform(df)

feature : combine all columns together (vector column)

In [ ]:
assembled.select("features").show(truncate=False)

+---------------------------------------------------------+
|features                                                 |
+---------------------------------------------------------+
|[918.0600000000087,2.863283199999908,295.39999999999986] |
|[917.3476881177097,3.5333236016106238,140.47154847112498]|
|[923.0400000000084,22.100967200000003,63.6999999999999]  |
|[920.5027511759195,5.190045360219378,211.20334124325618] |
|[921.1600000000036,2.863283199999954,136.49999999999991] |
|[915.300000000006,10.983375399999995,188.99999999999986] |
|[915.5988675132801,4.589632428699432,186.60669587586753] |
|[918.0700000000023,3.6462121999999924,271.5999999999999] |
|[920.080000000006,5.883152199999977,62.99999999999986]   |
|[915.0100000000116,6.576603599999951,195.89999999999992] |
|[919.650000000002,4.764682199999966,85.5]                |
|[915.6400000000099,8.030614600000042,265.7999999999999]  |
|[917.3900000000112,2.01324599999997,211.79999999999993]  |
|[920.8200000000083,3.310671200000037,16

In [ ]:
assembled.show(10)

+-----------------+------------------+----------------------+------------------+----------------------+------------------+---------------------+-----------------+---------------------+---------------------+--------------------+
| air_pressure_9am|      air_temp_9am|avg_wind_direction_9am|avg_wind_speed_9am|max_wind_direction_9am|max_wind_speed_9am|rain_accumulation_9am|rain_duration_9am|relative_humidity_9am|relative_humidity_3pm|            features|
+-----------------+------------------+----------------------+------------------+----------------------+------------------+---------------------+-----------------+---------------------+---------------------+--------------------+
|918.0600000000087| 74.82200000000041|                 271.1| 2.080354199999768|    295.39999999999986| 2.863283199999908|                  0.0|              0.0|    42.42000000000046|   36.160000000000494|[918.060000000008...|
|917.3476881177097| 71.40384263106537|    101.93517935618371|2.4430092157340217|    140.

Label : value that you want to predict (relative humidity 9am column) based on specified features.

In [ ]:
select_assembled=assembled.select("relative_humidity_9am","features")
select_assembled=select_assembled.withColumnRenamed("relative_humidity_9am","label")

In [ ]:
select_assembled.show(10)

+------------------+--------------------+
|             label|            features|
+------------------+--------------------+
| 42.42000000000046|[918.060000000008...|
|24.328697291802207|[917.347688117709...|
| 8.900000000000047|[923.040000000008...|
|12.189101868764444|[920.502751175919...|
| 92.41000000000044|[921.160000000003...|
|35.130000000000265|[915.300000000006...|
|10.657421657822635|[915.598867513280...|
| 80.47000000000041|[918.070000000002...|
| 29.58000000000045|[920.080000000006...|
|  88.6000000000012|[915.010000000011...|
+------------------+--------------------+
only showing top 10 rows



Goal : to train the model to predict "Relative Humidity 9am" based on the other features. so that when we give it new feature values, it can predict the label.

Training data is the part of the dataset used to teach the model.

Test data is the part you don't show the model during training — you use it to check how good the model is on new, unseen data.

In [ ]:
(trainData, testData) = select_assembled.randomSplit([0.8,0.2], seed = 13234 )

In [ ]:
trainData.count(),testData.count()

(846, 218)

In [ ]:
from pyspark.ml.regression import LinearRegression


In [ ]:
trainData.show()

+------------------+--------------------+
|             label|            features|
+------------------+--------------------+
| 6.090000000001012|[917.610000000009...|
| 6.290000000000094|[921.340000000008...|
| 6.929473038978267|[920.802725848539...|
|7.1300000000004005|[917.650000000007...|
| 7.320000000000401|[924.720000000002...|
| 7.370000000000948|[916.040000000011...|
| 7.490000000000099|[922.69000000001,...|
| 7.522056578279606|[922.273264482375...|
| 7.540000000000102|[922.040000000009...|
| 8.000000000001187|[920.400000000010...|
| 8.040000000000013|[920.320000000003...|
| 8.082800653859666|[922.928408958627...|
| 8.300000000000079|[915.610000000004...|
| 8.309953099773844|[922.902062345702...|
| 8.471749805902828|[920.632354382020...|
| 8.480000000001143|[919.500000000011...|
| 8.498335363231277|[915.914435739919...|
|  8.62000000000035|[921.220000000001...|
| 8.679999999999879|[915.710000000005...|
| 8.691746315441277|[918.846598183232...|
+------------------+--------------

In [ ]:
testData.show()

+------------------+--------------------+
|             label|            features|
+------------------+--------------------+
| 7.810000000000022|[923.020000000008...|
| 8.480000000001185|[919.700000000012...|
| 8.596749101632177|[922.254438037558...|
| 8.630765355780632|[922.593064124066...|
| 8.730000000000341|[918.300000000002...|
| 8.762945026170621|[921.211065935867...|
|  9.35000000000058|[921.020000000003...|
| 9.374424197076939|[919.477927109483...|
| 9.690133679405253|[920.791124402495...|
| 9.870100982481777|[922.221203584900...|
|10.018302278003464|[919.576336622174...|
|10.730000000000294|[920.020000000009...|
|10.753680105260125|[922.573065432606...|
|10.839536280499082|[918.694866756976...|
| 10.93387416268179|[920.277416872081...|
| 10.94000000000033|[922.600000000002...|
|10.940916402259122|[923.394623971261...|
|10.970000000000422|[923.780000000001...|
| 11.22000000000008|[926.550000000009...|
|11.279192121415758|[921.645839484382...|
+------------------+--------------

In [ ]:
lr = LinearRegression()


train model

In [ ]:
model1 = lr.fit(trainData)


In [ ]:
print("Coefficients: %s" % str(model1.coefficients))
print("Intercept: %s" % str(model1.intercept))

Coefficients: [-2.7961446567378982,0.3523476240817445,0.11223030704597362]
Intercept: 2584.400215975838


In [ ]:
trainingSummary = model1.summary
print("numIterations: %d" % trainingSummary.totalIterations)
print("objectiveHistory: %s" % str(trainingSummary.objectiveHistory))
trainingSummary.residuals.show()
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)

numIterations: 0
objectiveHistory: [0.0]
+-------------------+
|          residuals|
+-------------------+
|-21.715554454431228|
|-19.709336572567604|
|-20.337160212757773|
| -19.43712423042296|
| 2.5788591564326095|
|-29.081860136735337|
| -10.88846066687179|
|-11.808979759986524|
|-12.324233201744708|
|-15.464168774641344|
| -16.69305363582857|
| -9.647140371717652|
|-28.166007762757648|
|  -22.5043393197247|
|-14.987819256631855|
|-21.859440936958677|
|-27.165449064718118|
| -17.03573551469844|
|-30.230258042150083|
|-29.069568420122422|
+-------------------+
only showing top 20 rows

RMSE: 21.777495
r2: 0.252604


predict the model. make predictions on test data

In [ ]:
predictions = model1.transform(testData)

In [ ]:
predictions.show()

+------------------+--------------------+------------------+
|             label|            features|        prediction|
+------------------+--------------------+------------------+
| 7.810000000000022|[923.020000000008...|30.545222286118133|
| 8.480000000001185|[919.700000000012...| 36.55395074465514|
| 8.596749101632177|[922.254438037558...| 32.59420369788813|
| 8.630765355780632|[922.593064124066...|18.407987009486988|
| 8.730000000000341|[918.300000000002...| 28.78914599893824|
| 8.762945026170621|[921.211065935867...|25.432543681536572|
|  9.35000000000058|[921.020000000003...| 19.51997082440812|
| 9.374424197076939|[919.477927109483...|25.513216534817275|
| 9.690133679405253|[920.791124402495...|26.662780632353588|
| 9.870100982481777|[922.221203584900...|21.415939731176422|
|10.018302278003464|[919.576336622174...|25.343134255295354|
|10.730000000000294|[920.020000000009...| 37.63992095662343|
|10.753680105260125|[922.573065432606...|21.527689856635334|
|10.839536280499082|[918

In [ ]:
sc.stop()